In [ ]:
# Import relevant packages.
using DataFrames
using CSV
using Dates
using TimeZones
using StatsPlots

In [ ]:
heuristic_1_results = CSV.read("../data/heuristic_1_results.csv", DataFrame)
heuristic_2_results = CSV.read("../data/heuristic_2_results.csv", DataFrame)
withdraw_transactions_df = CSV.read("../data/lighter_complete_withdraw_txs.csv", DataFrame)
deposit_transactions_df = CSV.read("../data/lighter_complete_deposit_txs.csv", DataFrame);

In [ ]:
# Concat the results linked transactions we founds in the first and second heuristic
linked_transactions = vcat(heuristic_1_results, heuristic_2_results)

In [ ]:
# Environment settings for the notebook.
ENV["COLUMNS"]=10000
ENV["LINES"]=10;

In [ ]:
function string_to_datetime(datetime)
    ZonedDateTime(datetime, dateformat"yyyy-mm-dd HH:MM:SS ZZZ")
end
    

In [ ]:
function get_time_between_transactions(linked_transactions, withdraw_transactions_df, deposit_transactions_df)
    l = Array{Any}(undef, size(linked_transactions, 1))
    
    for (idx, row) in enumerate(eachrow(linked_transactions))
        idx_withdraw = findfirst(x -> x == row.withdraw_hash, withdraw_transactions_df[!, :hash])
        idx_desposit = findfirst(x -> x == row.deposit_hash, deposit_transactions_df[!, :hash])
        time_between_transactions = string_to_datetime(withdraw_transactions_df[idx_withdraw, :block_timestamp ]) - string_to_datetime(deposit_transactions_df[idx_desposit, :block_timestamp])
        l[idx] = time_between_transactions.value / (1000*60*60)
    end
    l
end

In [ ]:
time_between_transactions = get_time_between_transactions(linked_transactions, withdraw_transactions_df, deposit_transactions_df)

In [ ]:
# Histogram of the time between two linked transactions
histogram(time_between_transactions, bins=500, alpha=0.7, normalize = true, xlabel= "hours", label= false)

In [ ]:
function percentege(time_between_transactions, percentaje)
    time = sort(time_between_transactions)[trunc(Int, length(time_between_transactions) * percentaje )]
    "The $(percentaje * 100)% of the deposits stays in tornado cash less than $(round(time, digits=2)) hours"
end

In [ ]:
function hours_inside_tornado(time_between_transactions, hours)
    sorted_time_between_transactions = sort(time_between_transactions)
    transactions_made_before = findfirst(x -> x > hours, sorted_time_between_transactions) -1 
    "Trasactions that stay in tornado cash less than $(hours) hours: $(transactions_made_before) ($(round((transactions_made_before/ length(time_between_transactions)) * 100, digits = 2))%)"

end


In [ ]:
percentege(time_between_transactions, 0.95)

In [ ]:
percentege(time_between_transactions, 0.5)

In [ ]:
hours_inside_tornado(time_between_transactions, 1631 )